# Week 4 Implementation: Evolution Strategies for Non-Differentiable RL

**Authors:** Anirudh Bharadwaj, Shaurya Singhi, Sid Srivastava, Roberto Tamez  
**Date:** February 6, 2026  
**Course:** STAT 4830

This notebook demonstrates a working implementation of Evolution Strategies (ES) for sparse reward gridworld environments.

## Problem Setup

### Clear Problem Statement

**Goal:** Learn a policy π_θ that navigates from bottom-left to top-right in a gridworld with obstacles.

**Challenge:** Rewards are sparse (+1 at goal, 0 elsewhere), making gradient-based learning difficult.

**Approach:** Use parameter-space optimization via Evolution Strategies (ES), which does not require differentiable rewards.

### Mathematical Formulation

**Objective:**
$$\max_{\theta} J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T r_t \right]$$

**Evolution Strategies Gradient Estimate:**
$$\nabla_\theta J(\theta) \approx \frac{1}{N\sigma} \sum_{i=1}^N R(\theta + \sigma \epsilon_i) \cdot \epsilon_i$$

where $\epsilon_i \sim \mathcal{N}(0, I)$

**Update Rule:**
$$\theta_{t+1} = \theta_t + \alpha \cdot \nabla_\theta J(\theta_t)$$

### Data Requirements

**Environment:**
- State space: 64-dim (8×8 grid, one-hot encoded)
- Action space: 4 discrete actions {up, down, left, right}
- Episode length: max 50 steps
- Obstacles: 8 randomly placed

**Training Data:**
- Generated online through policy rollouts
- ES: 20 perturbations × 5 episodes = 100 episodes per iteration

### Success Metrics

1. **Success Rate:** % of episodes reaching goal (target: >30%)
2. **Average Return:** Mean cumulative reward
3. **Learning Stability:** Std dev across trials (lower is better)
4. **Sample Efficiency:** Iterations to reach threshold performance

## Implementation

In [1]:
# All required imports
import sys
sys.path.append('../src')  # Add src to path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Tuple, List, Dict

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 4)

print("Imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

Imports successful!
PyTorch version: 2.8.0
Device: cpu


### Environment Implementation

In [2]:
from model import GridWorld

# Test environment
env = GridWorld(size=8, n_obstacles=8, max_steps=50, seed=42)
state = env.reset()

print(f"Environment: {env.size}×{env.size} grid")
print(f"State shape: {state.shape}")
print(f"Action space: {env.n_actions} actions")
print(f"Start position: {env.start_pos}")
print(f"Goal position: {env.goal_pos}")
print(f"Number of obstacles: {len(env.obstacles)}")

# Visualize environment
env.render()

Environment: 8×8 grid
State shape: (64,)
Action space: 4 actions
Start position: (7, 0)
Goal position: (0, 7)
Number of obstacles: 8


<Figure size 800x800 with 1 Axes>

### Policy Network Implementation

In [3]:
from model import PolicyNetwork

# Create policy network
state_dim = env._get_state().shape[0]  # 64 for 8×8 grid
action_dim = env.n_actions  # 4
hidden_dim = 64
n_layers = 2

policy = PolicyNetwork(
    state_dim=state_dim,
    action_dim=action_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers
)

print(f"Policy Network:")
print(f"  Input dim: {state_dim}")
print(f"  Hidden dim: {hidden_dim}")
print(f"  Output dim: {action_dim}")
print(f"  Layers: {n_layers}")
print(f"  Total parameters: {sum(p.numel() for p in policy.parameters())}")

# Test forward pass
with torch.no_grad():
    state_tensor = torch.FloatTensor(state).unsqueeze(0)
    logits = policy(state_tensor)
    probs = F.softmax(logits, dim=-1)
    print(f"\nTest forward pass:")
    print(f"  Input shape: {state_tensor.shape}")
    print(f"  Output shape: {logits.shape}")
    print(f"  Action probs: {probs.squeeze().numpy()}")
    print(f"  Sum: {probs.sum().item():.6f}")

Policy Network:
  Input dim: 64
  Hidden dim: 64
  Output dim: 4
  Layers: 2
  Total parameters: 8580

Test forward pass:
  Input shape: torch.Size([1, 64])
  Output shape: torch.Size([1, 4])
  Action probs: [0.22937416 0.3139211  0.21942699 0.23727772]
  Sum: 1.000000


### Objective Function & Optimization Algorithm

**Objective function:** `evaluate_policy` computes $J(\theta) = \mathbb{E}[\sum r_t]$ by averaging episode returns under the current policy.

**Optimization algorithm:** `es_step` implements one step of Evolution Strategies — perturbing parameters, evaluating fitness via the objective, and estimating the gradient from the population.

In [4]:
def evaluate_policy(policy, env, n_episodes=5, max_steps=50):
    """Evaluate policy and return average reward."""
    total_reward = 0.0
    
    for _ in range(n_episodes):
        state = env.reset()
        episode_reward = 0.0
        done = False
        steps = 0
        
        while not done and steps < max_steps:
            action, _ = policy.get_action(state, deterministic=False)
            state, reward, done, _ = env.step(action)
            episode_reward += reward
            steps += 1
        
        total_reward += episode_reward
    
    return total_reward / n_episodes


def es_step(policy, env, N=20, sigma=0.05, n_eval_episodes=5, max_steps=50):
    """
    Single ES optimization step.
    
    Args:
        policy: PolicyNetwork to optimize
        env: Environment for evaluation
        N: Population size
        sigma: Noise scale
        n_eval_episodes: Episodes per perturbation
        max_steps: Max steps per episode
    
    Returns:
        gradient: Estimated gradient
        avg_reward: Average reward across population
    """
    # Get flattened parameters
    params = torch.cat([p.flatten() for p in policy.parameters()])
    n_params = params.shape[0]
    
    # Sample perturbations and evaluate
    perturbations = []
    rewards = []
    
    for i in range(N):
        # Sample perturbation
        epsilon = torch.randn(n_params)
        perturbations.append(epsilon)
        
        # Perturb parameters
        perturbed_params = params + sigma * epsilon
        
        # Set perturbed parameters
        offset = 0
        for p in policy.parameters():
            numel = p.numel()
            p.data = perturbed_params[offset:offset+numel].view_as(p)
            offset += numel
        
        # Evaluate
        reward = evaluate_policy(policy, env, n_eval_episodes, max_steps)
        rewards.append(reward)
    
    # Estimate gradient
    rewards = torch.tensor(rewards)
    perturbations = torch.stack(perturbations)
    
    # Standardize rewards for stability
    rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-8)
    
    gradient = (perturbations.T @ rewards) / (N * sigma)
    
    # Restore original parameters
    offset = 0
    for p in policy.parameters():
        numel = p.numel()
        p.data = params[offset:offset+numel].view_as(p)
        offset += numel
    
    return gradient, rewards.mean().item()


print("ES functions defined successfully!")

ES functions defined successfully!


### Key Parameters and Choices

**ES Hyperparameters:**
- Population size N = 20 (balance between gradient quality and computation)
- Noise scale σ = 0.05 (small enough for local search, large enough for exploration)
- Learning rate α = 0.01 (conservative to avoid instability)
- Evaluation episodes = 5 per perturbation (reduce variance)

**Design Choices:**
1. Reward standardization: Improves gradient stability
2. Parameter flattening: Easier gradient computation
3. Multiple evaluation episodes: Reduces environment stochasticity

### Basic Logging/Monitoring

In [5]:
class TrainingLogger:
    """Simple logger for tracking training progress."""
    
    def __init__(self):
        self.history = {
            'iteration': [],
            'avg_reward': [],
            'success_rate': [],
            'gradient_norm': []
        }
    
    def log(self, iteration, avg_reward, success_rate, gradient_norm):
        self.history['iteration'].append(iteration)
        self.history['avg_reward'].append(avg_reward)
        self.history['success_rate'].append(success_rate)
        self.history['gradient_norm'].append(gradient_norm)
    
    def plot(self):
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        
        axes[0].plot(self.history['iteration'], self.history['avg_reward'])
        axes[0].set_xlabel('Iteration')
        axes[0].set_ylabel('Average Reward')
        axes[0].set_title('Training Progress: Reward')
        axes[0].grid(True, alpha=0.3)
        
        axes[1].plot(self.history['iteration'], self.history['success_rate'])
        axes[1].set_xlabel('Iteration')
        axes[1].set_ylabel('Success Rate')
        axes[1].set_title('Training Progress: Success Rate')
        axes[1].grid(True, alpha=0.3)
        
        axes[2].plot(self.history['iteration'], self.history['gradient_norm'])
        axes[2].set_xlabel('Iteration')
        axes[2].set_ylabel('Gradient Norm')
        axes[2].set_title('Gradient Magnitude')
        axes[2].set_yscale('log')
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

print("Logger class defined!")

Logger class defined!


## Validation

### Test Cases with Results

In [6]:
# Test 1: Environment mechanics
print("Test 1: Environment Mechanics")
print("="*50)

env = GridWorld(size=4, n_obstacles=0, max_steps=20, seed=42)
env.start_pos = (0, 0)
env.goal_pos = (3, 3)

state = env.reset()
print(f"✓ Environment resets correctly")
print(f"  Start: {env.start_pos}, Goal: {env.goal_pos}")

# Test actions
actions = [1, 1, 1, 0, 0, 0]  # Right, Right, Right, Up, Up, Up
for i, action in enumerate(actions):
    state, reward, done, info = env.step(action)
    if i < len(actions) - 1:
        assert not done, "Should not be done yet"
    else:
        assert done, "Should reach goal"
        assert info['success'], "Should be successful"
        assert reward == 1.0, "Should get goal reward"

print(f"✓ Action execution works correctly")
print(f"✓ Goal detection works")
print(f"✓ Reward structure correct\n")

# Test 2: Policy network
print("Test 2: Policy Network")
print("="*50)

policy = PolicyNetwork(state_dim=16, action_dim=4, hidden_dim=32, n_layers=2)
state = torch.randn(1, 16)
logits = policy(state)

assert logits.shape == (1, 4), "Output shape incorrect"
print(f"✓ Forward pass shape correct: {logits.shape}")

action, log_prob = policy.get_action(state[0].numpy())
assert 0 <= action < 4, "Action out of bounds"
assert log_prob is not None, "Log prob should be returned"
print(f"✓ Action sampling works: action={action}")
print(f"✓ Log prob computed: {log_prob:.4f}\n")

# Test 3: ES gradient estimation
print("Test 3: ES Gradient Estimation")
print("="*50)

env = GridWorld(size=4, n_obstacles=0, max_steps=20, seed=42)
policy = PolicyNetwork(state_dim=16, action_dim=4, hidden_dim=32, n_layers=2)

gradient, avg_reward = es_step(policy, env, N=5, sigma=0.05, n_eval_episodes=2, max_steps=20)

n_params = sum(p.numel() for p in policy.parameters())
assert gradient.shape[0] == n_params, "Gradient shape incorrect"
print(f"✓ Gradient shape correct: {gradient.shape}")
print(f"✓ Average reward: {avg_reward:.4f}")
print(f"✓ Gradient norm: {gradient.norm().item():.4f}")

print("\n" + "="*50)
print("All tests passed! ✓")
print("="*50)

Test 1: Environment Mechanics
✓ Environment resets correctly
  Start: (0, 0), Goal: (3, 3)


AssertionError: Should reach goal

### Performance Measurements

Quick training run to validate learning.

In [7]:
# Quick training run (20 iterations)
print("Quick Training Run: 20 iterations")
print("="*50)

env = GridWorld(size=8, n_obstacles=8, max_steps=50, seed=42)
policy = PolicyNetwork(
    state_dim=64,
    action_dim=4,
    hidden_dim=64,
    n_layers=2
)

logger = TrainingLogger()
alpha = 0.01  # Learning rate
N = 20  # Population size
sigma = 0.05  # Noise scale

# Get flattened parameters
params = torch.cat([p.flatten() for p in policy.parameters()])

for iteration in range(20):
    # ES step
    gradient, avg_reward = es_step(
        policy, env,
        N=N,
        sigma=sigma,
        n_eval_episodes=5,
        max_steps=50
    )
    
    # Update parameters
    params = params + alpha * gradient
    
    # Set updated parameters
    offset = 0
    for p in policy.parameters():
        numel = p.numel()
        p.data = params[offset:offset+numel].view_as(p)
        offset += numel
    
    # Evaluate
    eval_rewards = []
    eval_successes = []
    for _ in range(10):
        state = env.reset()
        episode_reward = 0
        done = False
        steps = 0
        
        while not done and steps < 50:
            action, _ = policy.get_action(state, deterministic=True)
            state, reward, done, info = env.step(action)
            episode_reward += reward
            steps += 1
        
        eval_rewards.append(episode_reward)
        eval_successes.append(float(info['success']))
    
    success_rate = np.mean(eval_successes)
    eval_reward = np.mean(eval_rewards)
    grad_norm = gradient.norm().item()
    
    logger.log(iteration, eval_reward, success_rate, grad_norm)
    
    if iteration % 5 == 0:
        print(f"Iter {iteration:3d}: reward={eval_reward:6.3f}, success={success_rate:.2f}, grad_norm={grad_norm:.4f}")

print("\nTraining complete!")
logger.plot()

Quick Training Run: 20 iterations
Iter   0: reward= 0.000, success=0.00, grad_norm=405.1559
Iter   5: reward= 0.000, success=0.00, grad_norm=401.7718
Iter  10: reward= 0.000, success=0.00, grad_norm=404.9162
Iter  15: reward= 0.000, success=0.00, grad_norm=402.5747

Training complete!


/var/folders/xn/81b_v12s5t5317x0pksjtnxr0000gn/T/ipykernel_76756/3020063817.py:41: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Resource Monitoring

In [8]:
import time
import psutil
import os

print("Resource Usage Analysis")
print("="*50)

# Time one iteration
start_time = time.time()
start_memory = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024  # MB

gradient, avg_reward = es_step(
    policy, env,
    N=20,
    sigma=0.05,
    n_eval_episodes=5,
    max_steps=50
)

end_time = time.time()
end_memory = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024  # MB

iteration_time = end_time - start_time
memory_used = end_memory - start_memory

print(f"Single ES Iteration:")
print(f"  Time: {iteration_time:.2f} seconds")
print(f"  Memory delta: {memory_used:.2f} MB")
print(f"  Current memory: {end_memory:.2f} MB")
print(f"\nEstimated 100 iterations: {iteration_time * 100 / 60:.1f} minutes")

# Model size
n_params = sum(p.numel() for p in policy.parameters())
model_size_mb = n_params * 4 / 1024 / 1024  # 4 bytes per float32

print(f"\nModel Statistics:")
print(f"  Parameters: {n_params:,}")
print(f"  Size: {model_size_mb:.3f} MB")

Resource Usage Analysis
Single ES Iteration:
  Time: 0.65 seconds
  Memory delta: -17.25 MB
  Current memory: 126.58 MB

Estimated 100 iterations: 1.1 minutes

Model Statistics:
  Parameters: 8,580
  Size: 0.033 MB


### Example Outputs

Visualize learned policy behavior.

In [9]:
def visualize_policy(policy, env, n_episodes=3):
    """Visualize policy rollouts."""
    fig, axes = plt.subplots(1, n_episodes, figsize=(5*n_episodes, 5))
    if n_episodes == 1:
        axes = [axes]
    
    for ep, ax in enumerate(axes):
        state = env.reset()
        done = False
        steps = 0
        trajectory = [env.agent_pos]
        
        while not done and steps < 50:
            action, _ = policy.get_action(state, deterministic=True)
            state, reward, done, info = env.step(action)
            trajectory.append(env.agent_pos)
            steps += 1
        
        # Plot grid
        grid = np.zeros((env.size, env.size))
        
        # Mark obstacles
        for obs in env.obstacles:
            grid[obs] = -1
        
        # Mark trajectory
        for i, pos in enumerate(trajectory):
            if grid[pos] == 0:  # Don't overwrite obstacles
                grid[pos] = (i + 1) / len(trajectory)
        
        # Mark start and goal
        grid[env.start_pos] = 0.5
        grid[env.goal_pos] = 1.0
        
        im = ax.imshow(grid, cmap='RdYlGn', vmin=-1, vmax=1)
        ax.set_title(f"Episode {ep+1}\n{'Success' if info['success'] else 'Failed'} ({steps} steps)")
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Add markers
        ax.plot(env.start_pos[1], env.start_pos[0], 'bo', markersize=15, label='Start')
        ax.plot(env.goal_pos[1], env.goal_pos[0], 'g*', markersize=20, label='Goal')
        
        if ep == 0:
            ax.legend(loc='upper left', fontsize=10)
    
    plt.tight_layout()
    plt.show()

print("Visualizing learned policy behavior:")
visualize_policy(policy, env, n_episodes=3)

Visualizing learned policy behavior:


/var/folders/xn/81b_v12s5t5317x0pksjtnxr0000gn/T/ipykernel_76756/2797500800.py:48: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Edge Case Handling

In [10]:
print("Edge Case Testing")
print("="*50)

# Edge Case 1: Empty grid (should achieve high success)
print("\nTest 1: Empty grid (no obstacles)")
env_empty = GridWorld(size=8, n_obstacles=0, max_steps=50, seed=42)
success_count = 0
for _ in range(20):
    state = env_empty.reset()
    done = False
    steps = 0
    while not done and steps < 50:
        action, _ = policy.get_action(state, deterministic=True)
        state, reward, done, info = env_empty.step(action)
        steps += 1
    if info['success']:
        success_count += 1

print(f"  Success rate: {success_count/20:.2%} (expect >80%)")

# Edge Case 2: Dense obstacles
print("\nTest 2: Dense obstacles (15 obstacles on 8×8)")
env_dense = GridWorld(size=8, n_obstacles=15, max_steps=50, seed=42)
success_count = 0
for _ in range(20):
    state = env_dense.reset()
    done = False
    steps = 0
    while not done and steps < 50:
        action, _ = policy.get_action(state, deterministic=True)
        state, reward, done, info = env_dense.step(action)
        steps += 1
    if info['success']:
        success_count += 1

print(f"  Success rate: {success_count/20:.2%} (expect >10%)")

# Edge Case 3: Larger grid
print("\nTest 3: Larger grid (12×12)")
env_large = GridWorld(size=12, n_obstacles=12, max_steps=100, seed=42)

# Need to create new policy for different state dim
policy_large = PolicyNetwork(
    state_dim=144,  # 12×12
    action_dim=4,
    hidden_dim=64,
    n_layers=2
)

success_count = 0
for _ in range(20):
    state = env_large.reset()
    done = False
    steps = 0
    while not done and steps < 100:
        action, _ = policy_large.get_action(state, deterministic=False)
        state, reward, done, info = env_large.step(action)
        steps += 1
    if info['success']:
        success_count += 1

print(f"  Success rate (untrained): {success_count/20:.2%} (random baseline)")

print("\n" + "="*50)
print("Edge case testing complete!")

Edge Case Testing

Test 1: Empty grid (no obstacles)
  Success rate: 0.00% (expect >80%)

Test 2: Dense obstacles (15 obstacles on 8×8)
  Success rate: 0.00% (expect >10%)

Test 3: Larger grid (12×12)
  Success rate (untrained): 0.00% (random baseline)

Edge case testing complete!


## Documentation

### Key Design Decisions

1. **One-hot state encoding:** Simple and works for small grids. For larger grids, could use embedding.

2. **Reward standardization in ES:** Dividing by std helps with gradient stability when fitness values have different scales.

3. **Population size N=20:** Trade-off between gradient quality (want large N) and computation (want small N). 20 is reasonable for toy problems.

4. **Noise scale σ=0.05:** Too large causes divergence, too small causes slow learning. This value was found through trial.

5. **Multiple evaluation episodes:** Reduces variance from environment randomness (obstacle placement varies between resets).

### Known Limitations

1. **State representation:** One-hot encoding doesn't scale beyond ~20×20 grids (400 dimensions)

2. **Sample efficiency:** ES requires N×n_episodes evaluations per iteration (100 episodes for N=20, n_episodes=5)

3. **Hyperparameter sensitivity:** Performance varies with σ, α, N - no automatic tuning yet

4. **No parallelization:** ES perturbations are evaluated sequentially (could parallelize with multiprocessing)

5. **Simple policy network:** 2-layer MLP may lack capacity for complex tasks

### Debug/Test Strategies

**Debugging ES:**
- Print gradient norm (should decrease over time as we converge)
- Check if rewards are improving (sanity check)
- Visualize policy behavior every N iterations
- Test on empty grid first (should quickly reach ~100% success)

**Common Issues:**
1. Gradient exploding: Reduce σ or α
2. No learning: Increase σ or α, check reward function
3. High variance: Increase n_eval_episodes or N
4. Slow convergence: May need more iterations or different initialization

### Next Steps

1. **Debug environment test failure:** Investigate assertion error in goal detection test
2. **Run longer training:** 100+ iterations to verify ES convergence on 8×8 grid
3. **Hyperparameter tuning:** Grid search over σ, α, N
4. **Implement PPO training loop:** Build `train_ppo` in `src/utils.py` using the existing `ValueNetwork` and `get_action_batch` scaffolding
5. **Harder environment:** Test on `HarderGridWorld` (key-door multi-stage task)
6. **Parallelization:** Implement multiprocessing for ES evaluations
7. **Advanced variants:** Try Natural ES or CMA-ES

## Summary

This notebook demonstrates a working Evolution Strategies implementation for sparse reward RL:

✅ **Problem clearly defined:** Parameter optimization for gridworld navigation  
✅ **Mathematics formalized:** ES gradient estimator with explicit update rule  
✅ **Implementation complete:** Policy network, ES optimizer, training loop  
⚠️ **Validation partial:** ES pipeline runs end-to-end; environment test revealed a bug to investigate; policy has not yet converged in 20 iterations  
✅ **Resource monitoring:** ~0.65 seconds/iteration, ~127 MB memory, 8,580 parameters  
✅ **Documentation thorough:** Design decisions, limitations, debug strategies  

**Current Status:** The ES implementation runs correctly but has not demonstrated convergence on the 8×8 obstacle grid within 20 iterations. Longer training runs (100+ iterations) and hyperparameter tuning are the immediate next steps.

See `report.md` for the full project writeup.